# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [47]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#nltk for tokenizing/lemmatizing
import nltk
nltk.download(['punkt','wordnet'])



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
# load data from database
engine = create_engine('sqlite:///textmessages.db')
df = pd.read_sql_table('MessageTable',con=engine)

In [49]:
df[df.isnull().any(axis=1)]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7465,8365,NOTES: It mark as not enough information,None,direct,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9936,11186,My thoughts and prayers go out to all the live...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9937,11188,I m sorry for the poor people in Haiti tonight...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9938,11189,RT selenagomez UNICEF has just announced an em...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9939,11192,lilithia yes 5.2 magnitude earthquake hit mani...,None,social,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9940,11193,RT TheNewsBlotter RT caribnews On Call Interna...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9941,11195,Most Eureka homeowners won&#39 t collect on qu...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9942,11196,Haiti hit by largest earthquake in over 200 ye...,None,social,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9943,11197,RT pinkelephantpun Earthquake Relief Donate ht...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9944,11198,Praying that today s earthquake in #Haiti mini...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [50]:
X = df['message']
y = df.iloc[:,4:]

In [51]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [52]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
y.shape

(26345, 36)

In [54]:
X.shape

(26345,)

In [55]:
y.nunique()

related                   3
request                   2
offer                     2
aid_related               2
medical_help              2
medical_products          2
search_and_rescue         2
security                  2
military                  2
child_alone               1
water                     2
food                      2
shelter                   2
clothing                  2
money                     2
missing_people            2
refugees                  2
death                     2
other_aid                 2
infrastructure_related    2
transport                 2
buildings                 2
electricity               2
tools                     2
hospitals                 2
shops                     2
aid_centers               2
other_infrastructure      2
weather_related           2
floods                    2
storm                     2
fire                      2
earthquake                2
cold                      2
other_weather             2
direct_report       

#### Observation
'Related' category has 3 values, when these are supposed to be binary. Might eliminate rows that fall out of the binary classification. 'Child-alone' only has 1 value, meaning it won't provide much of anything to the ML process. Might eliminate as well.

In [56]:
#eliminate rows with null values
many_nans = y[y.isnull().any(axis=1)].index

y.drop(many_nans, inplace=True)
X.drop(many_nans, inplace=True)

In [57]:
print(y.shape, X.shape)

(26207, 36) (26207,)


In [58]:
#Further investigation of 'related' column
y['related'].unique()

array([ 1.,  0.,  2.])

In [59]:
y[y['related']==2.].shape

(192, 36)

In [60]:
#eliminate rows where 'related' category is 2 (non-binary)
non_binary_related = y[y['related']==2.].index

y.drop(non_binary_related, inplace=True)
X.drop(non_binary_related, inplace=True)

In [61]:
print(y.shape, X.shape)

(26015, 36) (26015,)


### 2. Write a tokenization function to process your text data

In [62]:
#this function will be used later as the tokenizer param in the CountVectorizer 
def tokenize(text):
    #split into tokens
    tokens = nltk.word_tokenize(text)
    
    #get to the ROOT of the words (lemmatize)
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token, pos='v').lower()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [63]:
# X[X.loc[X_train['message'].index].columns[2:]]

In [64]:
#custom transformer. still not certain if necessary

# class Grab_Categories(BaseEstimator, TransformerMixin):
#     def __init__(self, data):
#         self.data = data
    
#     def grab_cats(self, message):
#         mes_idx = message.index
        
#         return data[data.loc[data[mes_idx].columns[2:]]]
    
#     def fit(self, X):
#         return self
    
#     def transform(self, X):
#         X_cats = pd.Series(X).apply(self.grab_cats)
#         return pd.DataFrame(X_cats)

In [65]:
#works purely on message column
pipeline = Pipeline(
    [('vectorizer',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))]
)

# pipeline = Pipeline([
#     ('features',FeatureUnion[
#         #performs text tokenizing/vectorizing/TfidfTransform on messages
#         ('text_pipeline',Pipeline([
#             ('vectorizer',CountVectorizer(tokenizer=tokenize)),
#             ('tfidf',TfidfTransformer())
#         ])),
#         #grabs categories previously provided for messages
#         ('grab_categories',Grab_Categories(data=X))
#     ]),
#     ('clf',RandomForestClassifier(random_state=42))
# ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [67]:
y_train.shape

(20812, 36)

In [68]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [69]:
y_pred = pipeline.predict(X_test)

In [73]:
target_names = y.columns

In [74]:
classification_report(y_test, y_pred, target_names=target_names)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


'                        precision    recall  f1-score   support\n\n               related       0.77      0.93      0.84      3966\n               request       0.39      0.07      0.12       912\n                 offer       0.00      0.00      0.00        19\n           aid_related       0.46      0.19      0.27      2176\n          medical_help       0.17      0.00      0.00       409\n      medical_products       0.05      0.00      0.01       261\n     search_and_rescue       0.00      0.00      0.00       148\n              security       0.00      0.00      0.00        91\n              military       1.00      0.01      0.01       152\n           child_alone       0.00      0.00      0.00         0\n                 water       0.13      0.01      0.01       347\n                  food       0.19      0.01      0.02       590\n               shelter       0.17      0.00      0.01       472\n              clothing       1.00      0.03      0.06        69\n                 money

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.